In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.6 MB/s eta 0:00:00


In [ ]:
import re, json
import pdfplumber

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
def create_main_dic():

    L1_dic={"BidMonthYear": None,
            "Fleet":None, "Base":None,
            "Effective Date": None,
            "Through Date": None,
            "Pairings": [],
            "FTM": None,
            "TTL": None
                     }

    return L1_dic


def create_pairing_dic():
    L2_dic = {
        "ID": None,
        "Pairing Category": None,
        "F/O": None,
        "Effective Date": None,
        "Through Date": None,
        "Date Instances": [],
        "Duty Periods": [],
        "Days": None,
        "Credit": None,
        "Flight Time": None,
        "Time Away From Base": None,
        "International Flight Time": None,
        "NTE": None,
        "M$": None,
        "T/C": None
    }
    return L2_dic

def create_duty_periods_dic():

    L3_dic= {
        "Report": None ,
        "LEGS": [],
        "Release Time": None
    }

    return L3_dic

def create_legs_dic():

    L4_dic= {

        "Equipment": None,
        "Deadhead": None,
        "Flight Number": None,
        "Departure Station": None,
        "Arrival Station": None,
        "Departure Time": None,
        "Arrival Time": None,
        "Ground Time": None,
        "Meal Code": None,
        "Flight Time": None,
        "Accumulated Flight Time": None,
        "Duty Time": None,
        "D/C": None
    }
    return L4_dic

def create_master_dic():

    master_dic= {

        "data": []
    }
    return master_dic


In [ ]:
import re, json

def airline_pairing_processor(input_path, output_path):

    try:
        # Open and extract text from the PDF
        with pdfplumber.open(input_path) as pdf:
            raw_lines = []
            for page in pdf.pages:
                text = page.extract_text() or ""
                raw_lines.extend(text.splitlines())

    except Exception as e:
        print(f"Error reading PDF: {e}")
        return

    # raw_lines now contains a list of text lines exactly like your .dat workflow
    print("Successfully read PDF with", len(raw_lines), "lines")



    # Initiate a five level state structure using dictionaries

    Master_dic= create_master_dic()
    L1_dic= create_main_dic()
    L2_dic= create_pairing_dic()
    L3_dic= create_duty_periods_dic()
    L4_dic= create_legs_dic()

    # Iterate line-by-line thru the text data and identify unique sections and store them in their relevant sports.

    for line in raw_lines:
        if "1DSL" in line: #Pairing Header line
            L1_dic['BidMonthYear']=line[68:78].strip()
            L1_dic['Fleet']= line[35:38].strip()
            L1_dic['Base']= line[42:55].strip()
            L1_dic['Effective Date']= line[9:17].strip()
            L1_dic['Through Date']= line[23:31].strip()

        if "FTM" in line and "TTL" in line:

            pattern_with_codes = r"(FTM|TTL)-\s*(\d{1,}(?:,\d{1,})*:\d{2})"
            matches_with_codes = re.findall(pattern_with_codes, line)
            L1_dic['FTM']= matches_with_codes[0][1]
            L1_dic['TTL']= matches_with_codes[1][1]

            Master_dic["data"].append(L1_dic)
            L1_dic=create_main_dic()

        if "|" in line and "SU" not in line: # Indicates a calendar info line aka date instances. We are also avoiding the header line

            digit_pattern = r"(\d+)"
            L2_dic['Date Instances'].extend(re.findall(digit_pattern, line[109:129]))

        if "1DSL" not in line and "EFF" in line: # Start of Pairing

            substring= line[1:109] # Captures multiple lines, to expand the comment here
            pattern_extraction = r"EFF (\d{2}/\d{2}/\d{2}) THRU (\d{2}/\d{2}/\d{2}).*?(F/O)?\s*ID (\w+)\s+-\s+(\w+)(?:\s+\((\w+)\))?(\s*)?"
            matches_extraction = re.search(pattern_extraction, substring)
            if matches_extraction:
                eff_date, thru_date, fo_presence, id_value, category, optional_content, _ = matches_extraction.groups()
                L2_dic["Effective Date"]= eff_date
                L2_dic['Through Date']= thru_date
                L2_dic['F/O']= bool(fo_presence)
                L2_dic['ID']= id_value
                L2_dic['Pairing Category']= category + (f" ({optional_content})" if optional_content else "")



        if "RPT:" in line:

            pattern_report = r"RPT:\s*(\d+)"
            match_report = re.search(pattern_report, line[:109])
            L3_dic['Report']=match_report.group(1)

        if line[4:6].isdigit() and line[6].isalpha(): # Identify a leg entry

            L4_dic["Equipment"]= line[4:7]
            L4_dic["Deadhead"]= "DH" in line
            L4_dic["Flight Number"]= line[11:15].strip()
            L4_dic["Departure Station"]= line[16:19]
            L4_dic["Arrival Station"]= line[20:23]
            L4_dic["Meal Code"]= line[41].strip()
            L4_dic["Departure Time"]= line[24:28].strip()
            L4_dic["Arrival Time"]= line[29:33].strip()
            L4_dic["Ground Time"]= (lambda x: '0' if x == '.00' else x.replace('.', ':'))(line[35:40].strip())
            L4_dic["Flight Time"]= (lambda x: '0' if x == '.00' else x.replace('.', ':'))(line[48:53].strip())
            L4_dic["Accumulated Flight Time"]= (lambda x: '0' if x == '.00' else x.replace('.', ':'))(line[53:59].strip())
            L4_dic["Duty Time"]= (lambda x: '0' if x == '.00' else x.replace('.', ':'))(line[61:66].strip())
            L4_dic["D/C"]= (lambda x: '0' if x == '.00' else x.replace('.', ':'))(line[71:75].strip())

            L3_dic["LEGS"].append(L4_dic.copy()) # Append complete leg info into the list of dictionaries of the pairing dictionary
            L4_dic= create_legs_dic() # Cleare the leg dictionary for the next leg.


            continue

        if "RLS:" in line: # End of leg and RLS line

            pattern_report = r"RLS:\s*(\d+)"
            match_report = re.search(pattern_report, line[:109])
            L3_dic['Release Time']=match_report.group(1)


            L2_dic['Duty Periods'].append(L3_dic.copy())
            L3_dic= create_duty_periods_dic() # Clear duty periods dictionary for the next duty period


        if "DAYS-" in line: # This will also indicate an end of a pairing

            patterns = {
        "Days": r"DAYS-\s*(\d+)",
        "Credit": r"CRD-\s*([\d\.]+)",
        "Flight Time": r"FTM-\s*([\d\.:]+)",
        "Time Away From Base": r"TAFB-\s*([\d\.:]+)",
        "International Flight Time": r"INT-\s*([\d\.]+)",
        "NTE": r"NTE-\s*([\d\.]+)",
        "M$": r"M\$\-\s*([\d\.]+)",
        "T/C": r"T/C-\s*([\d\.]+)"
    }

            # Extracting the values using the regex patterns
            extracted_values = {}
            for label, pattern in patterns.items():
                match = re.search(pattern, line)
                if match:
                    extracted_values[label] = match.group(1)

            # Storing the extracted values in the L2_dic
            for key in extracted_values:

                if key in L2_dic:
                    L2_dic[key] = extracted_values[key]

            L1_dic["Pairings"].append(L2_dic.copy())
            L2_dic=create_pairing_dic() # Creates a fresh pairing dictionary for the next iterartion


    with open(output_path, "w") as json_file:
        json_file.write(json.dumps(Master_dic, indent=4))
        print('Pairing Successfuly completed and saved in ', str(output_path))

In [ ]:
airline_pairing_processor("p_ORD73701_upd.pdf", "ORD_output.json")

Successfully read PDF with 20498 lines


UnboundLocalError: cannot access local variable 'eff_date' where it is not associated with a value